In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from scipy import stats
import yaml
from astropy.coordinates import SkyCoord

from gammapy.data import DataStore
from regions import CircleSkyRegion
from gammapy.modeling import Fit
from gammapy.datasets import MapDataset
from gammapy.estimators import ExcessMapEstimator
from gammapy.modeling.models import Models, Model, SkyModel, FoVBackgroundModel
from gammapy.maps import MapAxis, WcsGeom, Map

import warnings
warnings.filterwarnings('ignore')

In [2]:
source = "Crab"

In [3]:
dataset = MapDataset.read(f'{source}/stacked.fits')
models = Models.read(f'{source}/standard_model.yml')
dataset.models = models